In [1]:
from molsberry.core import Pipeline, InputBlock, OutputBlock
from molsberry.core import SDFPathRep, PDBPathRep
from molsberry.core import LigandData, ProteinData, LocationData

from molsberry.modules.generic.pocket import (
    RDKitPocketIsolator, RDKitLigandPocketLocator, 
    PocketLocation, PocketLocationRep
)
from molsberry.modules.rdkit import RDKitMolRep

In [2]:
class ProteinIsolation(Pipeline):
    name = "protiso"
    display_name = "Protein Isolation Pipeline"
    def build(self):
        self.add_block(InputBlock(["ligand", "protein"]), "input")
        self.add_block(RDKitLigandPocketLocator(radius=10), "poclocator")
        self.add_block(RDKitPocketIsolator(), "isolator")
        self.add_block(OutputBlock(["pocket"]), "output")
        self.add_connection("input", "ligand", "poclocator", "ligand")
        self.add_connection("poclocator", "location", "isolator", "location")
        self.add_connection("input", "protein", "isolator", "protein")
        self.add_connection("isolator", "pocket", "output", "pocket")

In [3]:
from rdkit import Chem

rdlig = Chem.GetMolFrags(
    Chem.MolFromPDBFile(
        "/home/arazthexd/projects/002_sqm/data/targets/kguD.pdb"), asMols=True)[1]
ligdata = LigandData(RDKitMolRep(rdlig))
protdata = ProteinData(PDBPathRep("/home/arazthexd/projects/002_sqm/data/targets/kguD.pdb"))

In [4]:
pipeline = ProteinIsolation(base_dir="./ooooo")
out = pipeline.execute(ligand=ligdata, protein=protdata)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>       STARTED: Protein Isolation Pipeline         >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (poclocator) (RDKit) Ligand Pocket Locator]
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f90173fd900> <class 'molsberry.core.data.molecules.LigandData'>
<class 'molsberry.modules.rdkit.representations.RDKitMolRep'>

[Running Pipe Block: (isolator) (RDKit) Pocket Isolator]
<class 'molsberry.core.data.molecules.ProteinData'>
<molsberry.core.data.molecules.ProteinData object at 0x7f90173ffeb0> <class 'molsberry.core.data.molecules.ProteinData'>
<class 'molsberry.modules.rdkit.representations.RDKitProtRep'>
<class 'molsberry.core.data.generic.LocationData'>
<molsberry.core.data.generic.LocationData object at 0x7f90173fe8c0> <class 'molsberry.core.data.generic.LocationData'>
<class 'molsberry.modules.generic.pocket.locationrep.PocketLocationRe

In [5]:
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.drawMol3D(out["pocket"][0].get_representation_content(RDKitMolRep))

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [1]:
from molsberry.modules.rdkit import RDKitMolRep
from molsberry.modules.mopac import MOPACInputMolRep

from rdkit import Chem
from rdkit.Chem import rdDistGeom

mol = Chem.MolFromSmiles("CCCCCCC")
mol = Chem.AddHs(mol)
rdDistGeom.EmbedMolecule(mol)
rd_rep = RDKitMolRep(mol)
mopac_rep = MOPACInputMolRep.from_RDKitMolRep(rd_rep)

In [2]:
from molsberry.modules.cuby4.blocks import Cuby4MOPACEnergyCalculator
from molsberry.modules.cuby4.configs import Cuby4MOPACInterfaceConfig

block = Cuby4MOPACEnergyCalculator(
    interface_config=Cuby4MOPACInterfaceConfig(mozyme=True)
)
result = block.operate({
    "molecules": mopac_rep
})

In [3]:
result["energy"].content

-30.46117

In [5]:
print(mopac_rep.coordinates)

HETATM    1  C1  UNL     1      -2.588   0.838  -0.269  1.00  0.00           C  
HETATM    2  C2  UNL     1      -2.103  -0.161   0.729  1.00  0.00           C  
HETATM    3  C3  UNL     1      -1.367  -1.322   0.139  1.00  0.00           C  
HETATM    4  C4  UNL     1      -0.135  -0.899  -0.626  1.00  0.00           C  
HETATM    5  C5  UNL     1       0.861  -0.167   0.232  1.00  0.00           C  
HETATM    6  C6  UNL     1       2.030   0.189  -0.660  1.00  0.00           C  
HETATM    7  C7  UNL     1       3.106   0.934   0.095  1.00  0.00           C  
HETATM    8  H1  UNL     1      -2.297   0.660  -1.306  1.00  0.00           H  
HETATM    9  H2  UNL     1      -3.716   0.856  -0.270  1.00  0.00           H  
HETATM   10  H3  UNL     1      -2.252   1.872  -0.009  1.00  0.00           H  
HETATM   11  H4  UNL     1      -1.420   0.319   1.472  1.00  0.00           H  
HETATM   12  H5  UNL     1      -2.977  -0.597   1.288  1.00  0.00           H  
HETATM   13  H6  UNL     1  

In [1]:
import molsberry.core as core_
import molsberry.modules.generic as generic_
import molsberry.modules.cuby4 as cuby4_
import molsberry.modules.mopac as mopac_

class MyPipe(core_.Pipeline):
    name = "pipe"
    display_name = "My Pipe"
    def build(self):
        mopac_conf_prot = cuby4_.Cuby4MOPACInterfaceConfig(
            mozyme=True, keywords=["GEO-OK"]
        )
        mopac_conf_lig = cuby4_.Cuby4MOPACInterfaceConfig(
            mozyme=False, keywords=[]
        )
        mopac_conf_prot.config["job_cleanup"] = "no"
        
        self.add_block(core_.InputBlock(["ligand", "protein"]), "input")

        self.add_block(cuby4_.Cuby4MOPACEnergyCalculator(mopac_conf_lig), "lig_en")
        self.add_connection("input", "ligand", "lig_en", "molecules")
        # self.add_block(cuby4_.Cuby4MOPACEnergyCalculator(mopac_conf_prot), "prot_en")
        self.add_block(mopac_.MOPACSinglePointCalculator(mopac_.MOPACMozymeConfig()), "prot_en")
        # self.add_connection("input", "protein", "prot_en", "molecules")
        self.add_connection("input", "protein", "prot_en", "molecules")

        self.add_block(generic_.RDKitProteinLigandCombiner(), "combiner")
        self.add_connection("input", "ligand", "combiner", "ligands")
        self.add_connection("input", "protein", "combiner", "proteins")
        self.add_block(cuby4_.Cuby4MOPACEnergyCalculator(mopac_conf_prot), "com_en")
        self.add_connection("combiner", "complex", "com_en", "molecules")

        self.add_block(generic_.math.Adder(num_inputs=2), "pl_en_add")
        self.add_connection("lig_en", "energy", "pl_en_add", "num1")
        self.add_connection("prot_en", "energy", "pl_en_add", "num2")
        
        self.add_block(core_.OutputBlock(["lig_en", "prot_en", "pl_en"]), "output")
        self.add_connection("lig_en", "energy", "output", "lig_en")
        self.add_connection("prot_en", "energy", "output", "prot_en")
        self.add_connection("pl_en_add", "num_out", "output", "pl_en")

class MyPipe2(core_.Pipeline):
    name = "pipe2"
    display_name = "My Pipe 2"
    def build(self):
        conf_prot = cuby4_.Cuby4AMBERInterfaceConfig(
            home="/home/arazthexd/miniforge3/envs/sqmvscreen2"
        )
        conf_lig = cuby4_.Cuby4MOPACInterfaceConfig(
            mozyme=False
        )
        conf_qmmm = cuby4_.Cuby4QMMMInterfaceConfig(
            qm_config=conf_lig,
            mm_config=conf_prot
        )
        conf_qmmm.config["job_cleanup"] = "no"
        
        self.add_block(core_.InputBlock(["ligand", "protein"]), "input")

        self.add_block(generic_.RDKitLigandPocketLocator(radius=10), "poclocator")
        self.add_connection("input", "ligand", "poclocator", "ligand")

        self.add_block(generic_.RDKitPocketIsolator(), "isolator")
        self.add_connection("input", "protein", "isolator", "protein")
        self.add_connection("poclocator", "location", "isolator", "location")

        self.add_block(cuby4_.Cuby4QMMMEnergyCalculator(conf_qmmm), "qmmm")
        self.add_connection("input", "ligand", "qmmm", "qm_region")
        self.add_connection("isolator", "pocket", "qmmm", "nonqm_region")
        
        self.add_block(core_.OutputBlock(["lig_en", "prot_en", "pl_en"]), "output")
        self.add_connection("qmmm", "energy", "output", "pl_en")

pipe = MyPipe2()
result = pipe.execute(
    ligand=core_.MoleculeData(core_.SDFPathRep("./lig2.sdf")),
    protein=core_.MoleculeData(core_.PDBPathRep("./prot2.pdb"))
)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>                STARTED: My Pipe 2                 >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (poclocator) (RDKit) Ligand Pocket Locator]
<class 'molsberry.core.data.molecules.MoleculeData'>
<molsberry.core.data.molecules.MoleculeData object at 0x7f03d241bf10> <class 'molsberry.core.data.molecules.MoleculeData'>
<class 'molsberry.modules.rdkit.representations.RDKitMolRep'>

[Running Pipe Block: (isolator) (RDKit) Pocket Isolator]
<class 'molsberry.core.data.molecules.MoleculeData'>
<molsberry.core.data.molecules.MoleculeData object at 0x7f02a4556e30> <class 'molsberry.core.data.molecules.MoleculeData'>
<class 'molsberry.modules.rdkit.representations.RDKitProtRep'>
<class 'molsberry.core.data.generic.LocationData'>
<molsberry.core.data.generic.LocationData object at 0x7f02a45d0700> <class 'molsberry.core.data.generic.LocationData'>
<class 'molsberry.modules.generic.pocket.locationrep.PocketL

<class 'molsberry.core.data.molecules.MoleculeData'>
<molsberry.core.data.molecules.ProteinData object at 0x7f02a45d3910> <class 'molsberry.core.data.molecules.MoleculeData'>
<class 'molsberry.modules.openmm.representations.OpenMMInputMolRep'>
        _______  
       /\______\ 
      / /      / 
     / / Cuby /   Geometry optimization
     \/______/   
                 
Writting QM region geometry to qm_region.pdb
--------------------------------------------------------------------------------
Cycle: 0
Energy: -818.7611810943129
--------------------------------------------------------------------------------
Cycle: 1
Energy: -1178.050779238665
Convergence:
deltaE:   -359.2896   limit:   0.0060  NO
rmsGrad:    8.0297   limit:   0.6000  NO
maxGrad:  101.2076   limit:   1.2000  NO
--------------------------------------------------------------------------------
Cycle: 2
Energy: -1247.273167242454
Convergence:
deltaE:   -69.2224   limit:   0.0060  NO
rmsGrad:   10.9156   limit:   0.6000  N

AttributeError: 'NoneType' object has no attribute 'items'

In [21]:
result["prot_en"].get_representation_content()[0]

-6244.73833

In [11]:
import parmed

import molsberry.core as core_
import molsberry.modules.generic as generic_
import molsberry.modules.cuby4 as cuby4_
import molsberry.modules.mopac as mopac_
import molsberry.modules.openmm as omm_

pdb_rep = core_.PDBPathRep("/home/arazthexd/projects/002_sqm/data/targets/kguD.pdb")
omm_rep = omm_.OpenMMInputMolRep.from_PDBPathRep(pdb_rep)

struct: parmed.Structure = parmed.openmm.load_topology(omm_rep.topology, omm_rep.system, omm_rep.positions)

In [27]:
s = 0
for atom in struct.atoms:
    s += atom.charge
round(s)

2

In [13]:
struct.save("parms.parm7", overwrite=True)

In [16]:
struct.save("crd.pdb", overwrite=True)

In [6]:
omm_rep.system

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7f5786fb2b80> >

In [3]:
from collections import OrderedDict

a = OrderedDict()
a["c"] = 1

isinstance(a, dict)

True

In [1]:
import molsberry.modules.cuby4 as cuby4_
import molsberry.modules.generic as generic_
import molsberry.core as core_

class MyPipe2(core_.Pipeline):
    name = "pipe2"
    display_name = "My Pipe 2"
    def build(self):
        
        self.add_block(core_.InputBlock(["ligand", "protein"]), "input")

        self.add_block(generic_.RDKitLigandPocketLocator(radius=10), "poclocator")
        self.add_connection("input", "ligand", "poclocator", "ligand")

        self.add_block(generic_.RDKitPocketIsolator(), "isolator")
        self.add_connection("input", "protein", "isolator", "protein")
        self.add_connection("poclocator", "location", "isolator", "location")

        self.add_block(cuby4_.Cuby4AMBEREnergyCalculator(), "amberspc")
        self.add_connection("input", "ligand", "amberspc", "molecules")
        
        self.add_block(core_.OutputBlock(["lig_en", "prot_en", "pl_en"]), "output")
        self.add_connection("amberspc", "energy", "output", "lig_en")

pipe = MyPipe2()
result = pipe.execute(
    ligand=core_.MoleculeData(core_.SDFPathRep("./lig2.sdf")),
    protein=core_.MoleculeData(core_.PDBPathRep("./prot2.pdb"))
)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>                STARTED: My Pipe 2                 >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (amberspc) Cuby4-AMBER Single Point Calculator]
<class 'molsberry.core.data.molecules.MoleculeData'>
<molsberry.core.data.molecules.MoleculeData object at 0x7fde04382ce0> <class 'molsberry.core.data.molecules.MoleculeData'>
<class 'molsberry.modules.openmm.representations.OpenMMInputMolRep'>



[Running Pipe Block: (output) Output Block]

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<<                 ENDED: My Pipe 2                  <<
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [3]:
result["lig_en"] = 

{'lig_en': <molsberry.core.data.collections.BatchedData at 0x7fdcbd03f490>}

In [11]:
from rdkit import Chem

import numpy as np

from molsberry.modules.generic.pocket.locationrep import PocketLocation

In [118]:
rdprot = Chem.MolFromPDBFile(
    '/home/arazthexd/projects/002_sqm/data/targets/kguD.pdb')

lig = Chem.GetMolFrags(rdprot, asMols=True)[1]
loc = PocketLocation("ligand", ligand=lig, 
                     radius=4)
pos = rdprot.GetConformer().GetPositions()
remres = set()

allres = set(atom.GetPDBResidueInfo().GetResidueNumber() 
             for atom in rdprot.GetAtoms())
for atom in rdprot.GetAtoms():
    atom: Chem.Atom
    if atom.GetPDBResidueInfo().GetIsHeteroAtom():
        continue
    if loc.point_is_included(pos[atom.GetIdx()]):
        rn = atom.GetPDBResidueInfo().GetResidueNumber()
        remres.add(rn)
        if rn-1 in allres:
            remres.add(rn-1)
        if rn+1 in allres:
            remres.add(rn+1)

remres = sorted(remres)
if remres[0] == 1:
    terminal = [None]
else:
    terminal = ["N"]
for i, resnum in enumerate(remres[1:-1], start=1):
    prev_gap = (resnum - remres[i-1])
    next_gap = (remres[i+1] - resnum)
    
    if prev_gap == 1 and next_gap == 1:
        terminal.append(None)
        continue

    if prev_gap > 1 and next_gap > 1:
        terminal.append(None)
        continue

    if prev_gap > 1:
        terminal.append("N")
        continue

    if next_gap > 1:
        terminal.append("C")
        continue
if remres[-1] == max(allres):
    terminal.append(None)
else:
    terminal.append("C")

def nterm(emol: Chem.EditableMol, atom: Chem.Atom):
    aname = atom.GetPDBResidueInfo().GetName().strip()

    if aname in ["O", "C"]:
        pdbinfo = atom.GetPDBResidueInfo()
        pdbinfo.SetResidueName("ACE")
        emol.ReplaceAtom(atom.GetIdx(), atom)
        return
    
    emol.RemoveAtom(atom.GetIdx())
    return

def cterm(emol: Chem.EditableMol, atom: Chem.Atom):
    aname = atom.GetPDBResidueInfo().GetName().strip()

    if aname in ["N", "CA"]:
        pdbinfo = atom.GetPDBResidueInfo()
        pdbinfo.SetResidueName("NME")
        emol.ReplaceAtom(atom.GetIdx(), atom)
        return
    
    emol.RemoveAtom(atom.GetIdx())
    return

rdprote = Chem.EditableMol(rdprot)
rdprote.BeginBatchEdit()
for atom in rdprot.GetAtoms():
    atom: Chem.Atom
    rn = atom.GetPDBResidueInfo().GetResidueNumber()
    if rn in remres:
        if terminal[remres.index(rn)] == "N":
            nterm(rdprote, atom)
        if terminal[remres.index(rn)] == "C":
            cterm(rdprote, atom)
    else:
        rdprote.RemoveAtom(atom.GetIdx())
rdprote.CommitBatchEdit()

In [131]:
rdprotee = rdprote.GetMol()
Chem.SanitizeMol(rdprotee)
# rdprotee = Chem.AddHs(rdprotee, addCoords=True, addResidueInfo=True)

rdkit.Chem.rdmolops.SanitizeFlags.SANITIZE_NONE

In [132]:
print(Chem.MolToPDBBlock(rdprotee))

ATOM      1  C   ACE A  73      -0.524  -1.176  -8.965  1.00 97.98           C  
ATOM      2  O   ACE A  73      -1.061  -1.273 -10.068  1.00 97.51           O  
ATOM      3  N   VAL A  74      -1.237  -1.122  -7.838  1.00 97.22           N  
ATOM      4  CA  VAL A  74      -2.704  -0.980  -7.862  1.00 97.37           C  
ATOM      5  C   VAL A  74      -3.128   0.490  -7.904  1.00 97.59           C  
ATOM      6  O   VAL A  74      -4.084   0.835  -8.594  1.00 97.52           O  
ATOM      7  CB  VAL A  74      -3.374  -1.742  -6.702  1.00 96.88           C  
ATOM      8  CG1 VAL A  74      -3.115  -1.138  -5.327  1.00 93.94           C  
ATOM      9  CG2 VAL A  74      -4.890  -1.827  -6.914  1.00 95.22           C  
ATOM     10  N   GLY A  75      -2.413   1.358  -7.192  1.00 97.14           N  
ATOM     11  CA  GLY A  75      -2.634   2.800  -7.224  1.00 96.60           C  
ATOM     12  C   GLY A  75      -2.203   3.390  -8.564  1.00 96.73           C  
ATOM     13  O   GLY A  75  

In [133]:
#print(Chem.MolToPDBBlock(rdprotee))
pdbwriter = Chem.PDBWriter('new_poc.pdb')
pdbwriter.write(rdprotee)
pdbwriter.close()

In [144]:
from openmm import app, LangevinIntegrator
from pdbfixer import PDBFixer

pdb = PDBFixer("new_poc.pdb")
pdb.findMissingResidues()
pdb.findMissingAtoms()
pdb.addMissingAtoms()
pdb.addMissingHydrogens()
ff = app.ForceField("amber14-all.xml")
sys = ff.createSystem(pdb.topology)

In [146]:
simul = app.Simulation(pdb.topology, sys, LangevinIntegrator(300, 0.01, 0.002))
simul.context.setPositions(pdb.positions)
simul.minimizeEnergy()

In [163]:
with open("minimized.pdb", "w") as f:
    app.PDBFile.writeFile(pdb.topology, simul.context.getState(getPositions=True).getPositions(), f)
import nglview as nv
view = nv.show_file("minimized.pdb")
# view.clear_representations([0, 1])
# view.add_representation("line")

In [164]:
view

NGLWidget()

In [106]:
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.drawMol3D([rdprotee, lig])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [32]:
terminal

[None,
 'C',
 'N',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'C',
 'N',
 None,
 'C',
 'N',
 None,
 'C',
 'N',
 None,
 None,
 None,
 None,
 'C',
 'N',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'C',
 'N',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'C',
 'N',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'C',
 'N',
 None]

In [ ]:
rdprote = Chem.EditableMol(rdprot)
rdprote.BeginBatchEdit()

In [7]:
print(Chem.MolToPDBBlock(rdprot))

ATOM      1  N   MET A   1       8.873  -5.235 -35.015  1.00 84.74           N  
ATOM      2  CA  MET A   1       8.172  -5.611 -33.772  1.00 93.39           C  
ATOM      3  C   MET A   1       7.849  -4.341 -32.992  1.00 96.26           C  
ATOM      4  O   MET A   1       8.551  -3.352 -33.184  1.00 94.43           O  
ATOM      5  CB  MET A   1       9.042  -6.581 -32.962  1.00 84.77           C  
ATOM      6  CG  MET A   1       8.226  -7.740 -32.406  1.00 75.51           C  
ATOM      7  SD  MET A   1       9.207  -8.971 -31.486  1.00 73.85           S  
ATOM      8  CE  MET A   1      10.222  -9.707 -32.769  1.00 65.71           C  
ATOM      9  N   LYS A   2       6.810  -4.316 -32.161  1.00 95.64           N  
ATOM     10  CA  LYS A   2       6.549  -3.161 -31.291  1.00 97.33           C  
ATOM     11  C   LYS A   2       7.636  -3.043 -30.221  1.00 97.80           C  
ATOM     12  O   LYS A   2       8.190  -4.061 -29.803  1.00 97.08           O  
ATOM     13  CB  LYS A   2  